In [ ]:
import os
import json
import re
import unicodedata

CARPETA_FICHAS = "../assets/"

SECCIONES = {
    "indicaciones_terapeuticas": r"4\.1\..*?indicaciones\s*terapeuticas(.*?)(?=4\.2\.|5\.|$)",
    "posologia": r"4\.2\..*?posologia\s*y\s*forma\s*de\s*administracion(.*?)(?=4\.3\.|5\.|$)",
    "contraindicaciones": r"4\.3\..*?contraindicaciones(.*?)(?=4\.4\.|5\.|$)",
    "advertencias_precauciones": r"4\.4\..*?advertencias\s*y\s*precauciones\s*especiales\s*de\s*empleo(.*?)(?=4\.5\.|5\.|$)",
    "interacciones": r"4\.5\..*?interaccion\s*con\s*otros\s*medicamentos\s*y\s*otras\s*formas\s*de\s*interaccion(.*?)(?=4\.6\.|5\.|$)",
    "fertilidad_embarazo_lactancia": r"4\.6\..*?fertilidad.*?embarazo\s*y\s*lactancia(.*?)(?=4\.7\.|5\.|$)",
    "efectos_capacidad_conducir": r"4\.7\..*?efectos\s*sobre\s*la\s*capacidad\s*para\s*conducir\s*y\s*utilizar\s*maquinas(.*?)(?=4\.8\.|5\.|$)",
    "reacciones_adversas": r"4\.8\..*?reacciones\s*adversas(.*?)(?=4\.9\.|5\.|$)",
    "codigo_atc": r"codigo\s*atc.*?(\w{4}\w{3})",
    "fecha_revision": r"10\..*?fecha\s*de\s*la\s*revision\s*del\s*texto(.*?)(?=\n|$)"
}

def normalizar_texto(texto):
    # Eliminar números de página y encabezados
    texto = re.sub(r"\d+\s+de\s+\d+", "", texto)
    texto = re.sub(r"fic ha\s*tecnica", "", texto)
    
    # Convertir a minúsculas y quitar acentos
    texto = texto.lower()
    texto = ''.join(c for c in unicodedata.normalize('NFD', texto) 
                if unicodedata.category(c) != 'Mn')
    
    # Normalizar espacios y saltos de línea
    texto = re.sub(r"\s+", " ", texto)
    texto = re.sub(r"(?<!\n)\n(?!\n)", " ", texto)
    return texto.strip()

def extraer_seccion(texto, patron):
    match = re.search(patron, texto, re.DOTALL | re.IGNORECASE)
    if match:
        contenido = match.group(1).strip()
        # Limpiar subnumeraciones y listados
        contenido = re.sub(r"\d+\.\d+\.\d+", "", contenido)
        contenido = re.sub(r"[•▪]", "", contenido)
        return contenido
    return "No especificado"

def parsear_ficha(texto, nombre_archivo):
    texto_normalizado = normalizar_texto(texto)
    ficha_json = {"nombre": nombre_archivo.replace("_", " ").title()}
    
    for key, regex in SECCIONES.items():
        try:
            if key == "codigo_atc":
                match = re.search(regex, texto_normalizado)
                ficha_json[key] = match.group(1).strip().upper() if match else "No especificado"
            elif key == "fecha_revision":
                match = re.search(regex, texto_normalizado)
                if match:
                    fecha = match.group(1).strip().title()
                    fecha = re.sub(r"\b(\w+)", lambda m: m.group(1).capitalize(), fecha)
                    ficha_json[key] = fecha
                else:
                    ficha_json[key] = "No especificado"
            else:
                contenido = extraer_seccion(texto_normalizado, regex)
                ficha_json[key] = contenido if contenido else "No especificado"
        except Exception as e:
            print(f"Error en {key}: {str(e)}")
            ficha_json[key] = "Error en extracción"
    
    return ficha_json

def procesar_fichas(carpeta):
    fichas_procesadas = []
    
    for archivo in os.listdir(carpeta):
        if archivo.endswith(".txt"):
            nombre_medicamento = os.path.splitext(archivo)[0]
            ruta_completa = os.path.join(carpeta, archivo)
            try:
                with open(ruta_completa, "r", encoding="utf-8") as f:
                    texto = f.read()
                    ficha_json = parsear_ficha(texto, nombre_medicamento)
                    fichas_procesadas.append(ficha_json)
            except Exception as e:
                print(f"Error procesando {archivo}: {str(e)}")
    
    return fichas_procesadas

# Procesar y guardar
fichas_json = procesar_fichas(CARPETA_FICHAS)

with open("../assets/fichas_tecnicas.json", "w", encoding="utf-8") as f:
    json.dump(fichas_json, f, ensure_ascii=False, indent=4, sort_keys=True)

print(f"Procesadas {len(fichas_json)} fichas correctamente.")

Procesadas 152 fichas correctamente.
